# Cleaning the Auto Data Set

As mentioned in the introduction of the chapter on ETL, it is often the case in one off analyses that the ETL bleeds into the EDA and the boundary between the two is difficult to find. For the data we will use in this chapter, this is especially the case. We will load a CSV about *cars* from the University of California at Irvine (UCI) Machine Learning Data Repository. This particular data set is interested because it has a wide range of data types and thus will make the EDA more interesting. However, it comes from a CSV file and the data therein isn't exactly in the format we want.

Although we showed how we might use Python libraries to read CSVs in the previous chapter, we will use the Pandas library in this chapter. Pandas is the *de facto* library for working with data in Python as it replicates much of the functionality of DataFrames from R.

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 200)

One of the more peculiar behaviors of Pandas is that it seems to switch between nicely formatted HTML versions of data and raw terminal output without much warning. The settings above make those instances when the output is just raw terminal output more readable.

One of the nicer things about using Jupyter Notebooks is that you can use "magics" to access the command line/terminal. However, you need to be a bit careful because such commands are not necessarily cross-platform (ie, Windows-compatible).

We can view the first few lines of the data file by using `head`:

In [2]:
!head -n 5 "../resources/data/imports-85.data.csv"

3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,168.80,64.10,48.80,2548,dohc,four,130,mpfi,3.47,2.68,9.00,111,5000,21,27,13495
3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,168.80,64.10,48.80,2548,dohc,four,130,mpfi,3.47,2.68,9.00,111,5000,21,27,16500
1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.50,171.20,65.50,52.40,2823,ohcv,six,152,mpfi,2.68,3.47,9.00,154,5000,19,26,16500
2,164,audi,gas,std,four,sedan,fwd,front,99.80,176.60,66.20,54.30,2337,ohc,four,109,mpfi,3.19,3.40,10.00,102,5500,24,30,13950
2,164,audi,gas,std,four,sedan,4wd,front,99.40,176.60,66.40,54.30,2824,ohc,five,136,mpfi,3.19,3.40,8.00,115,5500,18,22,17450


The first thing we notice is that strings are not enclosed in quotation marks so if there are any numbers that should be interpreted as strings, we will need to be careful (zip codes, SKUs, social security numbers are the usual offenders here...anything that can start with meaningful leading zeros). The second thing we notice is that "?" is the missing value marker. Finally, we note that there is no header row. Luckily, a data dictionary provided with the CSV file tells us the names of the columns.

In [3]:
header = ["symbolizing", "normalized_losses", "make", "fuel_type", "aspiration", "doors", "body_style", "drive", "engine_location", "wheel_base", "length", "width", "height", "curb_weight", "engine_type", "cylinders", "engine_size", "fuel_system", "bore", "stroke", "compression_ratio", "horse_power","peak_rpm", "city_mpg", "highway_mpg", "price"]

We can take all this information and use `read_csv` in Pandas to create a DataFrame. We will supply `headers` as the names of the columns and we will tell Pandas that "?" is the missing value ("NA") marker:

In [4]:
autos = pd.read_csv( "../resources/data/imports-85.data.csv", names=header,na_values="?")

The first thing we want to do is use `info` to get a basic sense of the data and to look at the first few rows:

In [5]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
symbolizing          205 non-null int64
normalized_losses    164 non-null float64
make                 205 non-null object
fuel_type            205 non-null object
aspiration           205 non-null object
doors                203 non-null object
body_style           205 non-null object
drive                205 non-null object
engine_location      205 non-null object
wheel_base           205 non-null float64
length               205 non-null float64
width                205 non-null float64
height               205 non-null float64
curb_weight          205 non-null int64
engine_type          205 non-null object
cylinders            205 non-null object
engine_size          205 non-null int64
fuel_system          205 non-null object
bore                 201 non-null float64
stroke               201 non-null float64
compression_ratio    205 non-null float64
horse_power          203 non-

In [6]:
autos.head()

,symbolizing,normalized_losses,make,fuel_type,aspiration,doors,body_style,drive,engine_location,wheel_base,length,width,height,curb_weight,engine_type,cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horse_power,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


And now we want to start the rather long and sometimes tedious process of looking over all the variables. Usually, you will have some kind of domain knowledge that will help you tell if some of the values and ranges of values you observe are "janky". For this example, we'll have to rely on our common knowledge about cars...and where necessary, Wikipedia. When working on real world problems, you will often encounter variables that you don't understand and you'll need to hunt down the domain expert who does know.

To examine a variable, we can use `describe`:

In [7]:
autos["wheel_base"].describe()

count    205.000000
mean      98.756585
std        6.021776
min       86.600000
25%       94.500000
50%       97.000000
75%      102.400000
max      120.900000
Name: wheel_base, dtype: float64

`describe` prints out the basic descriptive statistics for the variable. In the case of a numeric variable, it will print out the usual parametric (mean, std) and non-parametric (min, 25%, 50%, 75%, max) descriptive statistics. It also prints out the *count* of non-NA values. You should make sure the value is what you think it should be.

It also shows you the *computer* type of the variable which can often be very important for working with the data if not necessarily interpreting it. For example, if we use `describe` on *make* which is a categorical variable we get:

In [8]:
autos["make"].describe()

count        205
unique        22
top       toyota
freq          32
Name: make, dtype: object

which is a very, very abbreviated summary of the data. It shows the *count*, the number of unique categories, and the most frequent category (the mode for categorical variables). This isn't always very useful. Sometimes we want to see *all* the categories and their frequencies. We can do this using `value_counts`:

In [9]:
autos["make"].value_counts()

toyota           32
nissan           18
mazda            17
mitsubishi       13
honda            13
volkswagen       12
subaru           12
volvo            11
peugot           11
dodge             9
mercedes-benz     8
bmw               8
plymouth          7
audi              7
saab              6
porsche           5
isuzu             4
alfa-romero       3
jaguar            3
chevrolet         3
renault           2
mercury           1
Name: make, dtype: int64

This time we have a complete distribution of the categories. Note that the computer type is now `int64`, this refers to the computer type of the frequency counts.

As mentioned before, you will build up a library of scripts and functions that you use regularly to ease the pain of repetitive tasks. Reviewing data is no exception. We can write a more comprehensive `describe` that works the way we would like with categorical variables. And does it for all the variables at once. Now, just because you automate the process doesn't mean you are relieved of having to read the output. You need to review each variable and this is a downside to automation...we can get lazy.

In [12]:
def describe_variables( df):
    for name in df.columns:
        print( "----------")
        if df[ name].dtype is np.dtype( 'O'):
            print( df[ name].value_counts())
        else:
            print( df[ name].describe())

In [13]:
describe_variables( autos)

----------
count    205.000000
mean       0.834146
std        1.245307
min       -2.000000
25%        0.000000
50%        1.000000
75%        2.000000
max        3.000000
Name: symbolizing, dtype: float64
----------
count    164.000000
mean     122.000000
std       35.442168
min       65.000000
25%       94.000000
50%      115.000000
75%      150.000000
max      256.000000
Name: normalized_losses, dtype: float64
----------
toyota           32
nissan           18
mazda            17
mitsubishi       13
honda            13
volkswagen       12
subaru           12
volvo            11
peugot           11
dodge             9
mercedes-benz     8
bmw               8
plymouth          7
audi              7
saab              6
porsche           5
isuzu             4
alfa-romero       3
jaguar            3
chevrolet         3
renault           2
mercury           1
Name: make, dtype: int64
----------
gas       185
diesel     20
Name: fuel_type, dtype: int64
----------
std      168
turbo     37
Na

There are still some problems here. The number of doors is indicated by words "four" and "two" even though they are clearly numeric values and ordered (2 doors is really twice 4 doors). The same is true of cylinders. These are things we will likely want to fix but will not for this chapter as it is not a tutorial on Pandas.

Otherwise, we can observe that some important variables have missin data (4 cars are missing prices, for example) and these are the kinds of things you're going to want to note and investigate. This was covered in the last chapter.